 **Build a model using transfer learning to classify a image is cat or dog.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # for mounting the drive

Mounted at /content/drive


In [ ]:
from zipfile import ZipFile

In [ ]:
with ZipFile('/content/drive/MyDrive/dogs-vs-cats.zip','r') as zipObj:
  # Extract all the content of zip file in current directory
  zipObj.extractall()

with ZipFile('train.zip','r') as zipObj:
  # Extract all the content of zip file in current directory
  zipObj.extractall()

with ZipFile('test1.zip','r') as zipObj:
  zipObj.extractall()

In [ ]:
# Create Directories
from os import makedirs
dataset_home='dataset_dogs_vs_cats/'
subdirs=['train/','test/']
for subdir in subdirs:
  # create label subdirectories
  labeldirs=['dogs/','cats/']
  for labldir in labeldirs:
    newdir=dataset_home+subdir+labldir
    makedirs(newdir,exist_ok=True)

Now change your dataset to fit the format

In [ ]:
from os import listdir
from shutil import copyfile
# create directories
from random import seed,random
seed(1)
# define ratio of pictures to use for validation
val_ratio=0.25 # means 0.25% images will be used for validation
# copy training dataset images into subdirectories
src_directory='train' # src is source
dataset_home='dataset_dogs_vs_cats/'
for file in listdir(src_directory):
  src=src_directory + '/' + file
  dst_dir='train/' # destination directory
  if random()<val_ratio:
    dst_dir='test/'
  if file .startswith('cat'):
    dst=dataset_home+dst_dir+'cats/'+file
    copyfile(src,dst)
  elif file.startswith('dog'):
    dst=dataset_home+dst_dir+'dogs/'+file
    copyfile(src,dst) # copyfile is using to copy file from one place to another

Prepare a model for the transfer learning

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
model=VGG16(include_top=False,input_shape=(224,224,3)) # this is the input shape of VGG16
# mark loaded layers as not trainable
for layer in model.layers:
  layer.trainable=False # means these layers are non trainable
# add new classifier layers
flat1=Flatten()(model.layers[-1].output) # now flatten out the final layers
class1=Dense(128,activation='relu',kernel_initializer='he_uniform')(flat1) # adding one dense layer
output=Dense(1,activation='sigmoid')(class1)
# define new model
model=Model(inputs=model.inputs,outputs=output) # now making a whole model
# compile model
opt=SGD(learning_rate=0.001,momentum=0.9) # define your optimizer,momentum gives us better conversion in gradient descent or optimizier , SGD is shocastic gradient descent
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy']) # compile the model, since this is binary class classification that's why we use this in loss

58889256/58889256 [==============================] - 1s 0us/step


Run the model using Generator

In [ ]:
datagen=ImageDataGenerator(featurewise_center=True) # Generator helps loading images into the ram in batches.
# specify imagenet mean values for centering
datagen.mean=[123.68,116.79,103.939] # it helps that your data remains centralized
# prepare iterator
train_it=datagen.flow_from_directory('dataset_dogs_vs_cats/train/', # flow_from_directory helps its takes all the images from your directory in batches and then it creates a generator or iterator, train_it is training iterator
  class_mode='binary',batch_size=64,target_size=(224,224))
test_it=datagen.flow_from_directory('dataset_dogs_vs_cats/test/', # test_it is testing iterator
  class_mode='binary',batch_size=64,target_size=(224,224))
# fit model
history=model.fit_generator(train_it,steps_per_epoch=len(train_it),
  validation_data=test_it,validation_steps=len(test_it),epochs=5,verbose=1)

Found 18697 images belonging to 2 classes.
Found 6303 images belonging to 2 classes.


<ipython-input-9-33f4369dcea2>:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(train_it,steps_per_epoch=len(train_it),


Epoch 1/5
 86/293 [=======>......................] - ETA: 2:23:22 - loss: 0.5140 - accuracy: 0.9529

Plot the loss and accuracy

In [ ]:
import matplotlib.pyplot as plt
import sys
pyplot.subplot(211)
pyplot.title('Cross Entropy Loss')
pyplot.plot(history.history['loss'],color='blue',label='train')
pyplot.plot(history.history['val_loss'],color='orange',label='test')
# plot accuracy
pyplot.subplot(212)
pyplot.title('Classification Accuracy')
pyplot.plot(history.history['accuracy'],color='blue',label='train')
pyplot.plot(history.history['val_accuracy'],color='orange',label='test')
# save file to plot
filename=sys.argv[0].split('/')[-1]
pyplot.savefig(filename+'_plot.png')

Save the model for future use

In [ ]:
model.save('final_model.h5')

Predict outputs from model

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
# load and prepare the image
# load the image
model=load_model('final_model.h5')
img=load_img('',target_size=(224,224))
# convert to array
img=img_to_array(img)
# reshape into a single sample with 3 channels
img=img.reshape(1,224,224,3)
# center pixel data
img=img.astype('float32')
img=img-[123.68,116.79,103.939]

In [ ]:
# 1 is for dogs and 0 is for cats
result=model.predict(img)
print(result[0])